In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

result_df = pd.read_csv('PaLM2input.csv', index_col=0)

In [2]:
result_df['GPT_input'] = result_df['GPT_input'].str.replace('minimum', 'min')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maximum', 'max')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('average', 'avg')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maxntprobnp', 'max ntprobnp')

In [3]:
result_df['GPT-Diagnoses'] = np.nan
result_df['GPT-Eval'] = np.nan

In [4]:
import vertexai

vertexai.init(project="n8n-copilot", location="us-central1")

In [5]:
# from vertexai.preview.language_models import TextGenerationModel
# parameters = {
#     "temperature": 0.2,
#     "max_output_tokens": 8192,
#     "top_p": 0.8,
#     "top_k": 40
# }
# model = TextGenerationModel.from_pretrained("medlm-medium")

In [6]:
from vertexai.preview.generative_models import GenerativeModel

multimodal_model = GenerativeModel("gemini-2.0-flash")

In [7]:
config = {
        "temperature": 0
    }

In [8]:
res = multimodal_model.generate_content(
        ['Give me a plan for a 4-day holiday in Mayrhofen'], generation_config=config)

In [9]:
print(res.candidates[0].content.parts[0].text)

Okay, here's a plan for a 4-day holiday in Mayrhofen, Austria, in May.  Keep in mind that May is a shoulder season, so some attractions might have limited hours or be closed.  It's always best to check specific opening times closer to your travel dates. This plan balances outdoor activities with relaxation and cultural experiences.

**Assumptions:**

*   You enjoy hiking and outdoor activities.
*   You are comfortable using public transportation (buses and trains).
*   You are interested in experiencing local culture.
*   You are arriving in Mayrhofen on Day 1 and departing on Day 4.

**Day 1: Arrival and Mayrhofen Exploration**

*   **Morning (Arrival):**
    *   Arrive at Innsbruck Airport (INN).  Take the train to Jenbach (approx. 30 minutes). From Jenbach, take the Zillertalbahn train to Mayrhofen (approx. 30 minutes).  Alternatively, you can arrange a private transfer from the airport directly to Mayrhofen.
    *   Check into your accommodation.  Consider hotels like ElisabethHote

In [10]:
prompt_no_example = '''You are an expert diagnostician machine for use by doctors. If the user input is not patient data, you politely decline the request. Please suggest diagnoses and conditions, followed by the evidence points supporting each diagnosis in the form of bullet points. Include previous diagnoses and pertinent information about the patient's medical history (if any). Pay close attention to all the history and investigations provided.  Put asterisks around the diagnoses to highlight them. Give each evidence points as a separate bullet point beneath the diagnosis. Include in your evidence points any relevant clinical scores that can be calculated from the information I have given. Do not explain the evidence points, only state them. For every diagnosis you list, if there are alternative differentials possible, state the most likely three in a bullet point beneath the evidence points (you do not need to state the evidence supporting them - you only need to do that for the main diagnoses). For the main diagnoses, give only confirmed diagnoses and evidence points that can be inferred solely based on the information I have given - do not use any other information. Only give me the information I have asked for - do not give me any other information. Do not give me any introductions or conclusions, safety instructions, or safety warnings. Use British English.

                                To illustrate how the information should be presented:

                                *MAIN DIAGNOSIS 1 AS HEADING*
                                evidence points to support MAIN DIAGNOSIS 1
                                The final bullet point is alternative differentials to consider: alternative 1, alternative 2, alternative 3

                                *MAIN DIAGNOSIS 2 AS HEADING*
                                evidence points to support MAIN DIAGNOSIS 2
                                The final bullet point is alternative differentials to consider: alternative 1, alternative 2, alternative 3

                                and so on...

Before finalising your answer check if you haven't missed any abnormal data points and hence any diagnoses or alternative differentials that could be made based on them. If you did, add them to your reply. If two diagnoses are commonly caused by the same underlying disease, have them under one header, which is the underlying disease.
'''

example = ''''''

patient_data = '''

Patient data:\n'''

In [ ]:
repeat = []
for i in tqdm(range(0,1000)):
    try:
        response = multimodal_model.generate_content(
            [prompt_no_example + example + patient_data + result_df.iloc[i]['GPT_input']], generation_config=config
        )
        hadm_id = result_df.index[i]
        result_df.loc[hadm_id, 'GPT-Diagnoses'] = response.candidates[0].content.parts[0].text
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        repeat.append(i)
        print(e)

 10%|▉         | 97/1000 [04:59<47:56,  3.19s/it]  

In [ ]:
for i in tqdm(repeat):
    try:
        response = multimodal_model.generate_content(
            [prompt_no_example + example + patient_data + result_df.iloc[i]['GPT_input']], generation_config=config
        )
        hadm_id = result_df.index[i]
        result_df.loc[hadm_id, 'GPT-Diagnoses'] = response.candidates[0].content.parts[0].text
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        print(e)

In [ ]:
result_df.to_csv('Temp0Gemini20Prod.csv')

In [15]:
result_df = pd.read_csv('Temp0Gemini20Prod.csv')

In [18]:
result_df['GPT-Diagnoses'].isna().sum()

1

In [19]:
result_df.loc[result_df['GPT-Diagnoses'].isna(),:]

,hadm_id,diagnoses,GPT_input,GPT-Diagnoses,GPT-Eval
240,21115325,1:Benign neoplasm of cerebral meninges\n2:Cere...,Blood report: \nThe patient stayed in the hosp...,NaN,NaN


In [22]:
repeat = []
for i in tqdm([240]):
    try:
        response = multimodal_model.generate_content(
            [prompt_no_example + example + patient_data + result_df.iloc[i]['GPT_input']], generation_config=config
        )
        hadm_id = result_df.index[i]
        result_df.loc[hadm_id, 'GPT-Diagnoses'] = response.candidates[0].content.parts[0].text
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        repeat.append(i)
        print(e)

100%|██████████| 1/1 [00:02<00:00,  2.00s/it]

Error happened at iteration i: 240
list index out of range


In [36]:
response = multimodal_model.generate_content(
            [prompt_no_example + example + patient_data + result_df.iloc[i]['GPT_input']], generation_config=config
        )

In [37]:
response

candidates {
  content {
  }
  finish_reason: RECITATION
  citation_metadata {
    citations {
      start_index: 987
      end_index: 1114
      title: "Comprehensive Handbook of Alcohol Related Pathology"
    }
  }
}
usage_metadata {
  prompt_token_count: 1962
  total_token_count: 1962
}

In [42]:
hadm_id = result_df.index[i]
result_df.loc[hadm_id, 'GPT-Diagnoses'] = 'Model declined to answer due to recitation'

In [43]:
result_df.to_csv('Temp0Gemini20Prod.csv')